In [8]:
import sys
sh_path = './scripts'
sys.path.insert(0, sh_path)

In [9]:
from tests import *
from regression import *
%matplotlib inline
# Para elegir que modelo usar para regresión, descomentar la línea con el modelo
# que se quiera usar y comentar las otras dos. Despues correr la celda
REGRESS_MODEL = "VAR" #o "VECM" o "OLS"

In [10]:
df_trimestral_crudo = pd.read_excel("https://github.com/mbarrena/tesis/blob/main/data/Data%20trimestral%201950%20a%202023%20con%20DUMMIES%20outliers%20(por%20trimestre).xlsx?raw=true")
display(df_trimestral_crudo.head(140))
display(df_trimestral_crudo.tail())


,año,trimestre,ipc_ajust,E,Ebc,pbird,impp_usa,Psoja_USA,Pmaíz_USA,Ptrigo_USA,tot,TOTfmi,D1,D2
0,1950,1ºTr,3.153126e-12,NaN,9.020000e-13,141630.000000,NaN,2.160000,1.166667,1.943333,NaN,NaN,0,0
1,1950,2ºTr,3.385298e-12,NaN,9.020000e-13,139320.000000,NaN,2.663333,1.320000,2.060000,NaN,NaN,0,0
2,1950,3ºTr,3.553683e-12,NaN,1.082330e-12,144669.000000,NaN,2.536667,1.440000,1.966667,NaN,NaN,0,0
3,1950,4ºTr,3.806171e-12,NaN,1.403200e-12,142804.000000,NaN,2.423333,1.396667,1.953333,NaN,NaN,0,0
4,1951,1ºTr,3.893961e-12,NaN,1.405173e-12,146731.000000,NaN,3.026667,1.580000,2.140000,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,1983,4ºTr,3.013218e-05,0.000002,1.946000e-06,356406.312947,60.935021,7.840000,3.156667,3.540000,NaN,96.39827,0,0
136,1984,1ºTr,4.732924e-05,0.000003,2.937333e-06,359573.567407,61.362723,7.603333,3.156667,3.463333,NaN,96.30234,0,0
137,1984,2ºTr,7.834598e-05,0.000004,4.426900e-06,366827.229115,62.029436,7.980000,3.340000,3.583333,NaN,96.38038,0,0
138,1984,3ºTr,1.367155e-04,0.000007,7.600833e-06,364304.377322,62.042016,6.513333,3.106667,3.383333,NaN,96.07735,0,0


,año,trimestre,ipc_ajust,E,Ebc,pbird,impp_usa,Psoja_USA,Pmaíz_USA,Ptrigo_USA,tot,TOTfmi,D1,D2
291,2022,4ºTr,12057.901663,162.437267,167.092767,715542.0,111.994742,13.966667,6.523333,9.100000,151.397002,99.653615,0,0
292,2023,1ºTr,14379.348389,192.295133,197.672200,726438.0,110.995728,14.833333,6.703333,8.566667,142.643274,100.255358,0,0
293,2023,2ºTr,17925.305868,231.982867,239.525000,705636.0,110.259612,14.500000,6.576667,8.016667,133.517789,100.212163,0,0
294,2023,3ºTr,22933.827291,312.495167,325.102767,721285.0,110.601380,14.000000,5.720000,7.336667,136.818441,100.063932,0,0
295,2023,4ºTr,32893.178687,448.259000,506.338867,704495.0,109.865265,12.933333,4.796667,6.756667,140.708694,99.721680,0,0


In [11]:
df_Arg = df_trimestral_crudo[['año', 'trimestre', 'ipc_ajust', 'E', 'Ebc', 'pbird', 'impp_usa', 'Psoja_USA', 'Pmaíz_USA', 'Ptrigo_USA', 'TOTfmi', "D1", "D2"]].copy()
df_Arg = renameColumnOfDataframe(df_Arg, "ipc_ajust", "ipc")
display(df_Arg)
display(df_Arg.tail(20))

,año,trimestre,E,Ebc,pbird,impp_usa,Psoja_USA,Pmaíz_USA,Ptrigo_USA,TOTfmi,D1,D2,ipc
0,1950,1ºTr,NaN,9.020000e-13,141630.0,NaN,2.160000,1.166667,1.943333,NaN,0,0,3.153126e-12
1,1950,2ºTr,NaN,9.020000e-13,139320.0,NaN,2.663333,1.320000,2.060000,NaN,0,0,3.385298e-12
2,1950,3ºTr,NaN,1.082330e-12,144669.0,NaN,2.536667,1.440000,1.966667,NaN,0,0,3.553683e-12
3,1950,4ºTr,NaN,1.403200e-12,142804.0,NaN,2.423333,1.396667,1.953333,NaN,0,0,3.806171e-12
4,1951,1ºTr,NaN,1.405173e-12,146731.0,NaN,3.026667,1.580000,2.140000,NaN,0,0,3.893961e-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,2022,4ºTr,162.437267,1.670928e+02,715542.0,111.994742,13.966667,6.523333,9.100000,99.653615,0,0,1.205790e+04
292,2023,1ºTr,192.295133,1.976722e+02,726438.0,110.995728,14.833333,6.703333,8.566667,100.255358,0,0,1.437935e+04
293,2023,2ºTr,231.982867,2.395250e+02,705636.0,110.259612,14.500000,6.576667,8.016667,100.212163,0,0,1.792531e+04
294,2023,3ºTr,312.495167,3.251028e+02,721285.0,110.601380,14.000000,5.720000,7.336667,100.063932,0,0,2.293383e+04


,año,trimestre,E,Ebc,pbird,impp_usa,Psoja_USA,Pmaíz_USA,Ptrigo_USA,TOTfmi,D1,D2,ipc
276,2019,1ºTr,39.002900,39.795533,692058.0,99.139008,8.560000,3.590000,5.266667,99.030840,0,0,2202.604388
277,2019,2ºTr,43.856967,43.776100,696592.0,99.743674,8.203333,3.713333,4.840000,99.120200,0,0,2447.440347
278,2019,3ºTr,50.444900,53.500833,697683.0,98.718370,8.316667,3.963333,4.373333,99.189390,0,0,2691.583155
279,2019,4ºTr,59.286800,59.828333,685852.0,98.376602,8.630000,3.746667,4.493333,99.281130,0,0,3044.533227
280,2020,1ºTr,61.396833,62.336300,654960.0,97.482747,8.636667,3.750000,4.873333,99.580729,0,0,3313.468834
281,2020,2ºTr,67.578233,68.608333,563921.0,94.065067,8.323333,3.216667,4.726667,99.741633,0,0,3521.560713
282,2020,3ºTr,73.227033,74.221667,624206.0,96.930661,8.800000,3.246667,4.603333,99.698682,0,0,3762.819357
283,2020,4ºTr,79.954567,81.256667,655278.0,97.614197,10.176667,3.790000,5.226667,100.103264,0,0,4152.067646
284,2021,1ºTr,88.464533,89.702767,675280.0,101.058166,12.266667,4.626667,5.723333,100.550051,0,0,4660.297721
285,2021,2ºTr,93.974167,94.655567,679772.0,104.659875,14.400000,5.740000,6.243333,100.648389,0,0,5228.309486


In [12]:
df_Arg59 = df_Arg.copy().iloc[36:].reset_index(drop=True)[['año', 'trimestre', 'ipc', 'E', 'Ebc', 'pbird', 'impp_usa', 'Psoja_USA', 'Pmaíz_USA', 'Ptrigo_USA', 'TOTfmi', "D1", "D2"]].copy()
display(df_Arg59)
display(df_Arg59.tail)

,año,trimestre,ipc,E,Ebc,pbird,impp_usa,Psoja_USA,Pmaíz_USA,Ptrigo_USA,TOTfmi,D1,D2
0,1959,1ºTr,2.413144e-11,6.690000e-12,6.669333e-12,177276.0,13.237287,2.046667,1.043333,1.736667,NaN,0,0
1,1959,2ºTr,3.074744e-11,8.556667e-12,8.317000e-12,182783.0,13.251340,2.106667,1.150000,1.743333,NaN,0,0
2,1959,3ºTr,3.511185e-11,8.408333e-12,8.414333e-12,166259.0,13.279444,1.976667,1.116667,1.723333,NaN,0,0
3,1959,4ºTr,3.704045e-11,8.313333e-12,8.275667e-12,178163.0,13.476177,1.970000,0.985667,1.780000,NaN,0,0
4,1960,1ºTr,3.935256e-11,8.293333e-12,8.297333e-12,186316.0,13.504281,1.996667,1.009667,1.800000,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,2022,4ºTr,1.205790e+04,1.624373e+02,1.670928e+02,715542.0,111.994742,13.966667,6.523333,9.100000,99.653615,0,0
256,2023,1ºTr,1.437935e+04,1.922951e+02,1.976722e+02,726438.0,110.995728,14.833333,6.703333,8.566667,100.255358,0,0
257,2023,2ºTr,1.792531e+04,2.319829e+02,2.395250e+02,705636.0,110.259612,14.500000,6.576667,8.016667,100.212163,0,0
258,2023,3ºTr,2.293383e+04,3.124952e+02,3.251028e+02,721285.0,110.601380,14.000000,5.720000,7.336667,100.063932,0,0


<bound method NDFrame.tail of       año trimestre           ipc             E           Ebc     pbird  \
0    1959      1ºTr  2.413144e-11  6.690000e-12  6.669333e-12  177276.0   
1    1959      2ºTr  3.074744e-11  8.556667e-12  8.317000e-12  182783.0   
2    1959      3ºTr  3.511185e-11  8.408333e-12  8.414333e-12  166259.0   
3    1959      4ºTr  3.704045e-11  8.313333e-12  8.275667e-12  178163.0   
4    1960      1ºTr  3.935256e-11  8.293333e-12  8.297333e-12  186316.0   
..    ...       ...           ...           ...           ...       ...   
255  2022      4ºTr  1.205790e+04  1.624373e+02  1.670928e+02  715542.0   
256  2023      1ºTr  1.437935e+04  1.922951e+02  1.976722e+02  726438.0   
257  2023      2ºTr  1.792531e+04  2.319829e+02  2.395250e+02  705636.0   
258  2023      3ºTr  2.293383e+04  3.124952e+02  3.251028e+02  721285.0   
259  2023      4ºTr  3.289318e+04  4.482590e+02  5.063389e+02  704495.0   

       impp_usa  Psoja_USA  Pmaíz_USA  Ptrigo_USA      TOTfmi  D1  D2

In [13]:
ChowTest(df_Arg59,19,21,y_vars=["ipc"],variables=removeInplace(df_Arg59.columns[2:-4],"tot"))

**Test de Chow:** idx before break 19, idx after break 21

**ipc**:

AttributeError: module 'chowtest' has no attribute 'ChowTest'